In [1]:
import brightway2 as bw
from pathlib import Path
from gsa_framework import LCAModel
from gsa_framework.validation import Validation
from gsa_framework.utils_setac_lca import *
from gsa_framework.utils import read_hdf5_array

Using environment variable BRIGHTWAY2_DIR for data directory:
/data/user/kim_a/Brightway3


In [2]:
path_base = Path('/data/user/kim_a/setac_gsa/')
path_merlin = path_base / "merlin"
filepath_scores_lsa_dict = path_merlin / "scores_lsa_dict.pickle"
filepath_params_yes_0 = path_merlin / "params_yes_0.pickle"
filepath_params_yes_6 = path_merlin / "params_yes_6.pickle"

with open(filepath_scores_lsa_dict, 'rb') as f:
    scores_lsa_dict = pickle.load(f)
    
with open(filepath_params_yes_0, 'rb') as f:
    params_yes_0 = pickle.load(f)
    
with open(filepath_params_yes_6, 'rb') as f:
    params_yes_6 = pickle.load(f)

In [3]:
def compute_influential_Y(scores_lca_dict, params_yes_0, params_yes_inf, iterations_validation=500):

    print("Number of influential parameters is {}".format(params_yes_inf.shape[0]))
    
    # LCA model
    bw.projects.set_current("GSA for setac")
    co = bw.Database("CH consumption 1.0")
    demand_act = [act for act in co if "Food" in act["name"]][0]
    demand = {demand_act: 1}
    method = ("IPCC 2013", "climate change", "GTP 100a")
    write_dir = path_base
    model = LCAModel(demand, method, write_dir)

    # Define some variables
    seed = 923458
    num_params = len(model)
    bin_min, bin_max = 2300, 3300

    default_amounts, uncertain_tech_params_where = get_amounts_means(
        model.lca.tech_params
    )
    static_mean_score = get_static_score(
        default_amounts, uncertain_tech_params_where, model.lca
    )
    validation = Validation(
        model,
        iterations=iterations_validation,
        seed=seed,
        default_x=default_amounts,
        write_dir=write_dir,
    )

    params_yes_inf_where = np.array([np.where(params_yes_0==p)[0][0] for p in params_yes_inf])
    influential_Y = validation.get_influential_Y_from_parameter_choice(
        params_yes_inf_where
    )
    validation.plot_histogram(base_y=validation.base_Y, influential_y=influential_Y)

In [4]:
# iterations_validation = 2000
# compute_influential_Y(scores_lsa_dict, params_yes_0, params_yes_6, iterations_validation)

# Validation after regression

In [5]:
def get_xgboost_params(path_model_dir, params_yes_0):
    path_model = path_model_dir / "model"
    path_params_yes_where = path_model_dir / "params_yes_where.pickle"

    with open(path_model, 'rb') as f:
        model = pickle.load(f)
    with open(path_params_yes_where, 'rb') as f:
        params_yes_where = pickle.load(f)

    fscore = model.get_score()
    fscore_max = max(fscore.values())
    for feature,fscore_val in fscore.items():
        fscore[feature] = fscore_val/fscore_max
    fscore_sorted = {k: v for k, v in sorted(fscore.items(), key=lambda item: item[1], reverse=True)}

    features_inf = np.array([int(i.replace('f','')) for i in fscore_sorted.keys()])
    importance_inf = np.array(list(fscore_sorted.values()))
    n_inf = len(fscore)
    params_yes_where_inf = params_yes_where[features_inf]

    importance_dict = {params_yes_where_inf[i]: importance_inf[i] for i in range(n_inf)}

    params_yes_xgboost = params_yes_0[params_yes_where_inf]
    
    return model, params_yes_xgboost

In [ ]:
# %%time
# seed = 78997
# path_model_dir = path_base / "regression" / "{}_model".format(seed)
# model, params_yes_xgboost = get_xgboost_params(path_model_dir, params_yes_0)

# params_yes_xgboost.sort()
# iterations_validation = 2000
# compute_influential_Y(scores_lsa_dict, params_yes_0, params_yes_xgboost, iterations_validation)

# Run MC with narrowed distributions

In [6]:
seed = 78997
path_model_dir = path_base / "regression" / "{}_model".format(seed)
model, params_yes_xgboost = get_xgboost_params(path_model_dir, params_yes_0)

iterations_validation = 20

In [7]:
bw.projects.set_current("GSA for setac")
co = bw.Database("CH consumption 1.0")
demand_act = [act for act in co if "Food" in act["name"]][0]
demand = {demand_act: 1}
method = ("IPCC 2013", "climate change", "GTP 100a")
write_dir = path_base
model = LCAModel(demand, method, write_dir)

array([122610, 143775, 156564, ..., 141644,  55871, 139785])